In [1]:
import pandas as pd
import numpy as np
import glob

## Import Datasets

In [2]:
#2010-2020 Age data
age2020 = pd.read_csv("C:\\Users\\Roy\\Desktop\\Project\\CountyPopData\\AgeGrpPopulations\\2010sAge\\20102020age.csv",
                     usecols = [1,2,5,6,39,42,45,48,51,54,57,60,63,66,69,72,
                                75,78,81,84,87,90])

age2020.columns = age2020.columns.str.lower()

#2000-2010 Age data (remove 2010 and use 2010 from 2010-2020 dataset)
#50 files (1/state)

path = r'C:\Users\Roy\Desktop\Project\CountyPopData\AgeGrpPopulations\2000sAge'
all_files = glob.glob(path + "/*.csv")

dfStore = []

for fileName in all_files:
    df = pd.read_csv(fileName, index_col = None, header = 0, encoding = "ISO-8859-1")
    dfStore.append(df)

age2010 = pd.concat(dfStore, axis = 0, ignore_index = True)
age2010.columns = age2010.columns.str.lower()
age2010 = age2010[["state","county","year","agegrp","tot_pop"]].copy()

#1990-1999 Age data
#10 files (1/year)

path = r'C:\Users\Roy\Desktop\Project\CountyPopData\AgeGrpPopulations\1990sAge'
all_files = glob.glob(path + "/*.csv")

dfStore = []

for fileName in all_files:
    df = pd.read_csv(fileName, index_col = None, header = None)
    dfStore.append(df)

age2000 = pd.concat(dfStore, axis = 0, ignore_index = True)
age2000.columns = ["year","fips","agegrp","racesex","ethncty","population"]

C:\Users\Roy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,39,42,45,48,51,54,57,60,63,66,69,72,75,78,81,84,87,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Fix 1990-1999 Dataset

In [3]:
age2000 = age2000.astype({'year':'str', 'fips':'str'})

age2000["State FIPS Code"] = age2000["fips"].str[:-3]
age2000["County FIPS Code"] = age2000["fips"].str[-3:]

del age2000['fips']
del age2000['racesex']
del age2000['ethncty']

age2000['year'] = '19' + age2000['year'].astype(str)

age2000 = age2000.astype({'year':'int', 'State FIPS Code':'int', 'County FIPS Code':'int'})

In [4]:
age19902000 = age2000.groupby(["State FIPS Code", "County FIPS Code", "year", "agegrp"]).population.sum().reset_index()
age19902000 = age19902000.groupby(["State FIPS Code", "County FIPS Code", "year"]).population.sum().reset_index()

In [5]:
age2000 = age2000.groupby(["State FIPS Code", "County FIPS Code", "year", "agegrp"]).population.sum().reset_index()

In [6]:
ageGroups = ["age<1","age14","age59","age1014","age1519","age2024",
             "age2529","age3034","age3539","age4044","age4549","age5054",
             "age5559","age6064","age6569","age7074","age7579","age8084",
             "age85plus"]

#Create column for each ageGroup in each county in each year
for i in range(0, len(ageGroups)):
    age19902000[ageGroups[i]] = age2000[age2000["agegrp"] == i]['population'].tolist()

age19902000['age14'] = age19902000['age14'] + age19902000['age<1']

del age19902000['age<1']

age19902000 = age19902000.rename(columns = {'age14':'age04'})

In [7]:
ageGroups = ["age04","age59","age1014","age1519","age2024",
             "age2529","age3034","age3539","age4044","age4549","age5054",
             "age5559","age6064","age6569","age7074","age7579","age8084",
             "age85plus"]

#Instead of counts use proportions of different age groups
for colName in ageGroups:
    age19902000[colName] = age19902000[colName] / age19902000['population']

In [9]:
#Fixed
#age19902000

## Fix 2000-2009 Dataset

In [10]:
yearMap = {1:0, 2:2000, 3:2001, 4:2002, 5:2003,
           6:2004, 7:2005, 8:2006, 9:2007, 10:2008,
           11:2009, 12:0, 13:0}

age2010 = age2010.replace({"year":yearMap})
age2010 = age2010[age2010["year"] != 0]
age2010 = age2010[age2010["agegrp"] != 99]

In [11]:
age20002009 = age2010.groupby(["state", "county", "year", "agegrp"]).tot_pop.sum().reset_index()
age20002009 = age20002009.groupby(["state", "county", "year"]).tot_pop.sum().reset_index()

In [12]:
age2010 = age2010.groupby(["state", "county", "year", "agegrp"]).tot_pop.sum().reset_index()

In [13]:
ageGroups = ["age<1","age14","age59","age1014","age1519","age2024",
             "age2529","age3034","age3539","age4044","age4549","age5054",
             "age5559","age6064","age6569","age7074","age7579","age8084",
             "age85plus"]

#Create column for each ageGroup in each county in each year (Same mapping and list used in 1990-1999 fix above)
for i in range(0, len(ageGroups)):
    age20002009[ageGroups[i]] = age2010[age2010["agegrp"] == i]['tot_pop'].tolist()

age20002009['age14'] = age20002009['age14'] + age20002009['age<1']
del age20002009['age<1']
age20002009 = age20002009.rename(columns = {'age14':'age04'})

In [16]:
ageGroups = ["age04","age59","age1014","age1519","age2024",
             "age2529","age3034","age3539","age4044","age4549","age5054",
             "age5559","age6064","age6569","age7074","age7579","age8084",
             "age85plus"]

#Instead of counts use proportions of different age groups
for colName in ageGroups:
    age20002009[colName] = age20002009[colName] / age20002009['tot_pop']

In [17]:
age20002009 = age20002009.rename(columns = {'state':'State FIPS Code', 'county':'County FIPS Code',
                                           'tot_pop':'population'})

In [19]:
#Fixed
#age20002009

## Fix 2010-2020 Dataset

In [20]:
yearMap = {1:0, 2:0, 3:2010, 4:2011, 5:2012, 6:2013, 7:2014,
           8:2015, 9:2016, 10:2017, 11:2018, 12:2019, 13:0, 14:2020}

age2020 = age2020.replace({"year": yearMap})
age2020 = age2020[age2020['year'] != 0]

In [21]:
columnNames = ["State FIPS Code", "County FIPS Code", "year", "population"] + ageGroups

age2020.columns = columnNames

In [36]:
#Change column dtypes to float
age2020 = age2020.astype({"population":"int","age04":"int","age59":"int","age1014":"int","age1519":"int","age2024":"int",
             "age2529":"int","age3034":"int","age3539":"int","age4044":"int","age4549":"int","age5054":"int",
             "age5559":"int","age6064":"int","age6569":"int","age7074":"int","age7579":"int","age8084":"int",
             "age85plus":"int"})

for colName in ageGroups:
    age2020[colName] = age2020[colName] / age2020['population']

In [38]:
#Fixed
#age2020

## Concatenate Fixed Datasets

In [40]:
ageData = pd.concat([age19902000, age20002009, age2020], axis = 0, ignore_index = True)

In [42]:
ageData.to_csv('Age_Fixed.csv', encoding = 'utf-8', index = False)